In [2]:
#Imports
import os
import pandas as pd
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm

In [4]:
casen = pd.read_csv(os.getcwd() + "/datasets/casen/CASEN 2009 con ingresos orig y ajust.csv", index_col=None, header=0, error_bad_lines=False)
casen.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (52,214,249,253,302,303,304,305,306,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,560,562,564,566,568,570,576) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,SEGMENTO,IDVIV,HOGAR,O,FOLIO,REGION,PROVINCIA,COMUNA,ZONA,ESTRATO,...,Y21E3X,Y21M4,Y21I4,Y21E4X,Y21M5,Y21I5,Y21E5X,Y21N6X,Y21M6,Y21I6
0,1101101,1,1,1,100011-K,1,11,1101,1,11011,...,,0,,,0,,,,0,
1,1101101,1,1,2,100011-K,1,11,1101,1,11011,...,,0,,,0,,,,0,
2,1101101,1,1,3,100011-K,1,11,1101,1,11011,...,,0,,,0,,,,0,
3,1101101,4,1,1,100041-1,1,11,1101,1,11011,...,,0,,,0,,,,0,
4,1101101,4,1,2,100041-1,1,11,1101,1,11011,...,,0,,,0,,,,0,


In [10]:
casen.dtypes['O20'] #¿Desde cuándo tiene su actual empleo? (año)

dtype('O')

In [ ]:
columns = ['ESC']
xS = gasolina.loc[:,columns]
y = (gasolina['G'] / gasolina['Pg'])

xS = sm.add_constant(xS)

sm.OLS(y,xS).fit().summary()